In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)


In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importing the dataset
dataset = pd.read_csv('/content/drive/MyDrive/class/9_deep_learning/churn_pred/Churn_Modelling.csv')


In [ ]:
dataset.head()

In [ ]:
x = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [ ]:
x

In [ ]:
y

In [ ]:
#Create dummy variables
pd.get_dummies(x["Geography"],drop_first=True)

In [ ]:

geography = pd.get_dummies(x["Geography"],drop_first=True)
gender=pd.get_dummies(x['Gender'],drop_first=True)

In [ ]:
## Concatenate the Data Frames
x = pd.concat([x,geography,gender],axis=1)

In [ ]:
x

In [ ]:
## Drop Unnecessary columns
x = x.drop(['Geography','Gender'],axis=1)

In [ ]:
x.head()

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2, random_state = 0)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler


In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
# Part 2 - Now let's make the ANN!

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU
from tensorflow.keras.layers import Dropout

In TensorFlow, the **Sequential class** is a fundamental building block for creating neural network models. It represents a linear stack of layers, where each layer in the stack is connected to the next layer. This allows you to define a neural network by stacking layers on top of each other in a sequential manner.<br><br>
In TensorFlow, the **Dense layer** is a commonly used layer type for building neural networks. It represents a fully connected layer where each neuron is connected to every neuron in the previous layer. The Dense layer is responsible for performing a matrix multiplication between the input data and its weights, followed by an element-wise activation function.<br><br>
In neural networks, **dropout** is a regularization technique used to prevent overfitting. The Dropout layer randomly sets a fraction of input units to 0 at each update during training time, which helps to prevent the neural network from relying too heavily on any specific input features. Dropout can be applied to intermediate layers of a neural network, forcing the network to learn more robust and generalizable representations.

In [ ]:
# Initialising the ANN
classifier = Sequential()

In [ ]:
# Adding the input layer 

classifier.add(Dense(units=11,activation='relu'))

In [ ]:
# Adding the first hidden layer

classifier.add(Dense(units=7,activation='relu'))
classifier.add(Dropout(0.2))

In [ ]:
# Adding the second hidden layer
classifier.add(Dense(units=6,activation='relu'))
classifier.add(Dropout(0.25))

In [ ]:
# Adding the output layer
classifier.add(Dense(units=1,activation='sigmoid'))

In [ ]:
# compile

classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

By default, Adam optimizer uses a learning rate of 0.001

In [ ]:
# if you want to give your own learning rate, 
# classifier.compile(optimizer='opt',loss='binary_crossentropy',metrics=['accuracy'])
# import tensorflow
# opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)

In [ ]:
# model_history = classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=1000)

**Early Stopping** <br>
[keras documentation for early stopping](https://keras.io/api/callbacks/early_stopping/)

In [ ]:
# early stopping
import tensorflow as tf

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [ ]:
model_history = classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,epochs=1000,callbacks=early_stopping)

In [ ]:
# list all data in history

print(model_history.history.keys())

In [ ]:

# summarize history for accuracy
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results

y_pred = classifier.predict(x_test)


In [ ]:
y_pred 

In [ ]:
y_pred >= 0.5

In [ ]:
y_pred = (y_pred >= 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [ ]:
score

In [ ]:
# get weights
classifier.get_weights()